<a href="https://colab.research.google.com/github/AmBha21/sparta-lab/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#imports
from google.colab import drive
drive.mount('/content/drive')

import re
from transformers import BertTokenizer, BertForMaskedLM
import torch
import torch.nn.functional as F

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# load BERT large cased model from HuggingFace
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


This is getting the baseline BERT parameters

In [13]:
# Example test sentences with [MASK]
test_sentences = [
    "In its economic policy, government should [MASK] taxes.",
    "[MASK] is the biggest issue in the US that needs immediate attention.",
    "In regards with immigration, government should [MASK] borders.",
    "Abortion should be [MASK]"
]

# Tokenize and predict
# Assuming device is defined as 'cuda' or 'cpu'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
aggregated_probabilities = []
for sentence in test_sentences:
    inputs = tokenizer.encode(sentence, return_tensors="pt")
    inputs = inputs.to(device)  # Move inputs to the same device as the model
    mask_token_index = torch.where(inputs == tokenizer.mask_token_id)[1]

    token_logits = model(inputs).logits
    mask_token_logits = token_logits[0, mask_token_index, :]

    # Get the top 5 tokens and their indices
    top_5_tokens = torch.topk(mask_token_logits, 5, dim=1)
    top_5_indices = top_5_tokens.indices[0].tolist()
    top_5_values = top_5_tokens.values[0]

    # Apply softmax to convert logits to probabilities
    probabilities = F.softmax(top_5_values, dim=0).tolist()
    temp_words = []

    print(f"Original Sentence: {sentence}")
    for index, token in enumerate(top_5_indices):

        word = tokenizer.decode([token])
        temp_words.append(word)
        probability = probabilities[index] * 100  # Convert to percentage
        new_sentence = sentence.replace(tokenizer.mask_token, word)
        print(f"Predicted Sentence: {new_sentence} (Certainty: {probability:.2f}%)")
    print("\n")
    aggregated_probabilities.append(list(zip(probabilities, temp_words)))

print(aggregated_probabilities)

Original Sentence: In its economic policy, government should [MASK] taxes.
Predicted Sentence: In its economic policy, government should reduce taxes. (Certainty: 39.50%)
Predicted Sentence: In its economic policy, government should raise taxes. (Certainty: 18.90%)
Predicted Sentence: In its economic policy, government should lower taxes. (Certainty: 15.98%)
Predicted Sentence: In its economic policy, government should avoid taxes. (Certainty: 13.70%)
Predicted Sentence: In its economic policy, government should levy taxes. (Certainty: 11.91%)


Original Sentence: [MASK] is the biggest issue in the US that needs immediate attention.
Predicted Sentence: this is the biggest issue in the US that needs immediate attention. (Certainty: 55.82%)
Predicted Sentence: it is the biggest issue in the US that needs immediate attention. (Certainty: 32.83%)
Predicted Sentence: that is the biggest issue in the US that needs immediate attention. (Certainty: 7.66%)
Predicted Sentence: aids is the bigges

In [ ]:
test_sentences = [
    "In its economic policy, government should [MASK] taxes.",
    "[MASK] is the biggest issue in the US that needs immediate attention.",
    "In regards with immigration, government should [MASK] borders.",
    "Abortion should be [MASK]"
]



In [3]:
left_wing_path = '/content/drive/My Drive/sparta lab/transcripts/left_videos.txt'

with open(left_wing_path, "r") as fp:
    lines = fp.readlines()  # Read all lines into a list

portion_length = len(lines)  # Calculate 1/50th of the total number of lines
partial_text = lines[:portion_length]  # Extract the first 1/8th portion

# Split the partial text into individual lines/sentences
text = [line.strip() for line in partial_text if line.strip()]

In [4]:
#text[:10]

In [ ]:
inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [ ]:
#inputs

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
#inputs.keys()

In [ ]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
mask_arr

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
selection[:5]

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
#inputs.input_ids

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
input("Press Enter to continue...")

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 1

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

In [ ]:
# Example test sentences with [MASK]
test_sentences = [
    "In its economic policy, government should [MASK] taxes.",
    "[MASK] is the biggest issue in the US that needs immediate attention.",
    "In regards with immigration, government should [MASK] borders.",
    "Abortion should be [MASK]"
]

# Tokenize and predict
# Assuming device is defined as 'cuda' or 'cpu'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

for sentence in test_sentences:
    inputs = tokenizer.encode(sentence, return_tensors="pt")
    inputs = inputs.to(device)  # Move inputs to the same device as the model
    mask_token_index = torch.where(inputs == tokenizer.mask_token_id)[1]

    token_logits = model(inputs).logits
    mask_token_logits = token_logits[0, mask_token_index, :]

    top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

    print(f"Original Sentence: {sentence}")
    for token in top_5_tokens:
        word = tokenizer.decode([token])
        new_sentence = sentence.replace(tokenizer.mask_token, word)
        print(f"Predicted Sentence: {new_sentence}")
    print("\n")

